In [1]:
from google.colab import drive

# Google Drive'ı bağla
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Gerekli kütüphaneleri yükle
!pip install transformers datasets torch evaluate

In [4]:
# Dosya yollarını ayarla
train_file_path = '/content/drive/MyDrive/Colab Notebooks/Bert-Base/final_train_data_v2.json'
dev_file_path = '/content/drive/MyDrive/Colab Notebooks/Bert-Base/final_dev_data_v2.json'

In [5]:
import json

# Veri setlerini yükle
with open(train_file_path, 'r', encoding='utf-8') as f:
    train_data = json.load(f)

with open(dev_file_path, 'r', encoding='utf-8') as f:
    dev_data = json.load(f)

print("Eğitim Verisinden Bir Örnek:")
print(json.dumps(train_data['data'][0], indent=4, ensure_ascii=False))  # Eğitim setinden bir örnek

print("\nDoğrulama Verisinden Bir Örnek:")
print(json.dumps(dev_data['data'][0], indent=4, ensure_ascii=False))  # Doğrulama setinden bir örnek


Eğitim Verisinden Bir Örnek:
{
    "paragraphs": [
        {
            "context": "Rollo'nun gelişinden önce popülasyonları Picardy'den veya 'Franklar' olarak nitelendirilen Île-de-France'den farklı değildi. Daha önceki Viking yerleşimcileri 880'lerde gelmeye başlamıştı ancak doğudaki koloniler (Roumois ve Pays de Caux) arasında alçak Seine vadisi çevresinde ve batıdaki Cotentin Yarımadası'ndaki koloniler arasında bölündü ve hemen hemen hiçbir yabancı yerleşimci olmadan nüfusun aynı kaldığı geleneksel pagii ile ayrıldılar.  Normandiya ve Atlantik kıyılarının bir kısmına baskın düzenleyen ve yerleşen Rollo'nun muhalifleri Norveçliler, İskandinav-İskoçyalılar, Orkney Vikingler, muhtemelen İsveçliler ve Norveç kontrolündeki İngiliz Danelaw'lardan Anglo Danimarkalıları içermektedir. ",
            "qas": [
                {
                    "answers": [
                        {
                            "answer_start": 0,
                            "text": "Rollo"
                

In [6]:
from datasets import DatasetDict, Dataset

# Veri setini düzenleyen fonksiyon
def prepare_data(data):
    contexts, questions, answers = [], [], []
    for item in data['data']:
        for paragraph in item['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                contexts.append(context)
                questions.append(qa['question'])
                answers.append({
                    'text': qa['answers'][0]['text'],
                    'answer_start': qa['answers'][0]['answer_start']
                })
    return contexts, questions, answers

# Eğitim ve doğrulama verilerini hazırlayın
train_contexts, train_questions, train_answers = prepare_data(train_data)
dev_contexts, dev_questions, dev_answers = prepare_data(dev_data)

# Hugging Face Dataset formatına dönüştürme
train_dataset = Dataset.from_dict({
    "context": train_contexts,
    "question": train_questions,
    "answers": train_answers
})
dev_dataset = Dataset.from_dict({
    "context": dev_contexts,
    "question": dev_questions,
    "answers": dev_answers
})

dataset = DatasetDict({
    "train": train_dataset,
    "validation": dev_dataset
})

In [7]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Model ve Tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def preprocess_data(examples):

     tokenized_examples = tokenizer(
         examples["question"],
         examples["context"],
         max_length=384,
         truncation=True,
         padding="max_length",
         return_offsets_mapping=True
     )

     start_positions = []
     end_positions = []

     for i, offsets in enumerate(tokenized_examples["offset_mapping"]):
         answer = examples["answers"][i]
         start_char = answer["answer_start"]
         end_char = start_char + len(answer["text"])

         # Offset içindeki token başlangıç ve bitiş pozisyonlarını bulma
         start_token = None
         end_token = None
         for j, (start, end) in enumerate(offsets):
             if start <= start_char < end:
                 start_token = j
             if start < end_char <= end:
                 end_token = j

         # Pozisyonları ayarlama
         if start_token is None or end_token is None:
             start_token, end_token = 0, 0

         start_positions.append(start_token)
         end_positions.append(end_token)

     tokenized_examples["start_positions"] = start_positions
     tokenized_examples["end_positions"] = end_positions
     return tokenized_examples

# Veri setini işleme
tokenized_datasets = dataset.map(
    preprocess_data,
    batched=True,
    remove_columns=["context", "question", "answers"]
)


Map:   0%|          | 0/14221 [00:00<?, ? examples/s]

Map:   0%|          | 0/1330 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# Eğitim parametrelerini ayarla
output_dir = '/content/drive/MyDrive/Colab Notebooks/Bert-Base/results'

training_args = TrainingArguments(
    output_dir=output_dir,          # Model çıktı dosyalarının kaydedileceği klasör
    evaluation_strategy="steps",    # Değerlendirme sıklığı (adım bazlı)
    eval_steps=500,                 # Kaç adımda bir değerlendirme yapılacağı
    learning_rate=5e-5,             # Öğrenme oranı
    per_device_train_batch_size=16, # Her cihaz için eğitim batch boyutu
    per_device_eval_batch_size=16,  # Her cihaz için değerlendirme batch boyutu
    num_train_epochs=9,             # Eğitim boyunca epoch sayısı
    weight_decay=0.01,              # Ağırlık azaltma (regularization)
    save_steps=500,                 # Model kaydedilecek adım sayısı
    save_total_limit=2,             # Maksimum kaç model kaydedileceği
    logging_dir="/content/drive/MyDrive/Colab Notebooks/Bert-Base/logs",  # Log dosyalarının kaydedileceği klasör
    logging_steps=100,              # Kaç adımda bir log kaydedileceği
    load_best_model_at_end=True,    # En iyi modeli eğitim sonunda yükler
    push_to_hub=False               # Hugging Face Hub'a yükleme seçeneği (opsiyonel)
)

# Trainer nesnesini tanımlayın
trainer = Trainer(
    model=model,                          # Kullanılacak model
    args=training_args,                   # Eğitim parametreleri
    train_dataset=tokenized_datasets["train"],    # Eğitim veri seti
    eval_dataset=tokenized_datasets["validation"], # Doğrulama veri seti
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Early Stopping eklendi

)

# Model eğitimi
trainer.train()

print("Model eğitimi tamamlandı!")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,1.958700,2.182731
1000,1.463500,1.918780
1500,1.468700,1.765224
2000,0.976700,1.976807
2500,0.972300,1.664403
3000,0.682100,1.775610
3500,0.670500,1.847664
4000,0.484800,2.087718


Model eğitimi tamamlandı!


In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Checkpoint yolunu belirtin
checkpoint_path = r"C:\Users\admin\Desktop\Bert-Base\results\checkpoint-2500"



# Tokenizer ve modeli yükle
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)  # Checkpoint'teki tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(checkpoint_path).to("cuda")  # Checkpoint'teki model

# Test verisi
context = """Osman Bey 1258 yılında Söğüt’te doğdu. Osman Bey 1 Ağustos 1326’da Bursa’da hayatını kaybetmiştir.
1281 yılında Osman Bey 23 yaşında iken Ahi teşkilatından olan Şeyh Edebali’nin kızı Malhun Hatun ile evlendi
.Bu evlilikten daha sonra Osmanlı Devleti’nin başına geçecek olan Orhan Gazi doğdu.
1281 yılında Osman Beyin babası Ertuğrul Bey 90 yaşında  vefat etmiştir.1326’da Osman Bey, Bursa’yı kuşattı.
Fakat Osman beyin rahatsızlanması üzerine kuşatmaya Orhan Bey devam etti. Bursa alındıktan sonra başkent yapılmıştır.
Osman Gazi son yıllarında yaşının ilerlemesi ve gut hastalığı yüzünden beylik idaresini oğlu olan Orhan Bey'e bırakmıştı.
Osmanlı Beyliğinin ilk fethettiği ada İmralı Adasıdır. İmralı Adası 1308 yılında Osman Bey tarafından alınmıştır.
İlk Osmanlı parası Osman Bey tarafından bakır olarak akçe adı ile 1324 yılında bastırılmıştır.
Osmanlı Beyliğinin ilk başkenti Söğüttür.Osmanlı tarihinde ilk savaş, 1284 yılında Bizans tekfurlarıyla yapılan Ermeni Beli savaşıdır.
Osman Beyin ele geçirdiği ilk kale 1285 yılında fethedilen Kolca Hisar Kalesi’dir.
Osmanlı beyliğinin ilk kadısı Osman Bey döneminde atanan Dursun Fakih’tir.
Osman Bey 1288 yılında Karacahisarı fethetti. Osman Bey 1299 yılında Bilecik'i fethetti.
Osman Gazi, babası Ertuğrul Gazi'den yaklaşık 4.800 kilometrekare olarak devraldığı Osmanlı toprağını oğlu Orhan Gazi'ye 16.000 kilometrekare olarak devretmiştir.
Osman Bey'in vefatı sonrası yerine Orhan Bey geçti."""

question = "1285 yılında hangi kale fethedilmiştir?"

# Soruyu ve bağlamı birleştir ve tokenleştir
inputs = tokenizer(
    question,
    context,
    return_tensors="pt",
    max_length=512,
    truncation=True,
    padding="max_length"
).to("cuda")

# Modelden çıkış al
outputs = model(**inputs)

# Başlangıç ve bitiş pozisyonlarını alın
start_index = outputs.start_logits.argmax(dim=-1).item()
end_index = outputs.end_logits.argmax(dim=-1).item()

# Cevabı çözümleme
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index : end_index + 1])
)

# Çıktıyı yazdır
print(f"Soru: {question}")
print(f"Cevap: {answer}")


Soru: 1285 yılında hangi kale fethedilmiştir?
Cevap: ilk kale


In [13]:
from transformers import AutoModel, AutoTokenizer

model_name = "bert-base-uncased"  # Model ismini buraya yaz
model = AutoModel.from_pretrained(model_name, cache_dir="C:/Users/admin/Desktop/Bert-Base/results/deneme")
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="C:/Users/admin/Desktop/Bert-Base/results/deneme")


c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
!cp -r /content/drive/MyDrive/Colab\ Notebooks/Bert-Base/results/checkpoint-2500 /content/drive/MyDrive/Colab\ Notebooks/Bert-Base/results/checkpoint-2500-copy


In [18]:
from transformers import AutoTokenizer

# Eğitim sırasında kullanılan tokenizer'ı yükle
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Eksik dosyaları checkpoint-4000'e kaydet
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/Bert-Base/results/checkpoint-2500')


('/content/drive/MyDrive/Colab Notebooks/Bert-Base/results/checkpoint-2500/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Bert-Base/results/checkpoint-2500/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Bert-Base/results/checkpoint-2500/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Bert-Base/results/checkpoint-2500/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Bert-Base/results/checkpoint-2500/tokenizer.json')